In [1]:
# gptname = 'gpt2' #small
gptname = 'gpt2-xl'
enablenews = True
colab = True
devicename = 'cuda'
lrmult = 0.1
accumn = 10
if not colab:
    filepath = '.'
else:
    filepath = '/content/drive/MyDrive/bio'
    ! pip3 install transformers
    from google.colab import drive
    drive.mount('/content/drive')

import sys
sys.path.append(filepath + '/utils')
sys.path.append(filepath + '/dataset')

     |████████████████████████████████| 2.2MB 12.2MB/s 
     |████████████████████████████████| 870kB 26.3MB/s 
     |████████████████████████████████| 3.3MB 50.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=91d3dcaf51c7007f52e78f772f9e8b1a8fff11c6430bb7f6fc8e58748fcaae27
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
Mounted at /content/drive


In [2]:
import argparse, json, os, pickle, random, time, numpy as np, transformers, torch

from transformers import GPT2Config, GPT2LMHeadModel,AdamW, GPT2Tokenizer
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tnrange, tqdm

from utils import SummarizationDataset, FineTuningDataset, get_tokenizer, generate_sample, sample_seq, set_seed, top_k_top_p_filtering

In [3]:
#please change default arguments if needed

parser = argparse.ArgumentParser()

parser.add_argument("--num_train_epochs",default=1, type=int,  help="no of epochs of training")

parser.add_argument("--seed",default=42, type=int,  help="seed to replicate results")
parser.add_argument("--num_workers",default=2, type=int,  help="num of cpus available")
parser.add_argument("--device",default=torch.device(devicename), help="torch.device object")
parser.add_argument("--model_dir",default= filepath + '/weights', type=str,  help="path to save trained model")
parser.add_argument("--root_dir",default= filepath + '/bignews/gpt2_1024_data', type=str, help="location of json dataset.")
parser.add_argument("--ids_file",default= filepath + '/bignews/ids.json', type=str, help="location of train, valid and test file indexes")
args = parser.parse_args([])

In [4]:
if enablenews:
  strnews = 'ye'
else:
  strnews = 'no'

model_file = os.path.join(args.model_dir,  'model_bio_{}_lr{}_accum{}_{}news.bin'.format(
          gptname, int(lrmult*100), accumn, strnews))
config_file = os.path.join(args.model_dir, 'config_bio_{}_lr{}_accum{}_{}news.json'.format(
          gptname, int(lrmult*100), accumn, strnews))

config = GPT2Config.from_json_file(config_file)
tokenizer = get_tokenizer(gptname)
model = GPT2LMHeadModel(config)
state_dict = torch.load(model_file, map_location=args.device)
_ = model.load_state_dict(state_dict)
_ = model.eval()
_ = model.to(args.device)

In [5]:
valid_data = FineTuningDataset(filepath + '/dataset', tokenizer, mode='valid', addnews=enablenews, length=10)
print(valid_data.mode)
print(valid_data.len)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
valid
10


In [6]:
for i in range(10):
  idx = valid_data[i]['sum_idx']
  article = valid_data[i]['article'][:idx+1].tolist()
  summary = valid_data[i]['article'][idx+1:][:100].tolist()

  print('new_article', end='\n\n')
  print(tokenizer.decode(article + tokenizer.encode(valid_data[i]['name'])), end='\n\n')
  print('actual_summary', end='\n\n')
  print(tokenizer.decode(summary, skip_special_tokens=True), end='\n\n')
  print("Generated Summary: \n")
  generated_text = sample_seq(
      model, article + tokenizer.encode(valid_data[i]['name']), 100, args.device, 
      #temperature=2, top_k=5, top_p=0.5
      temperature=3, top_k=5, top_p=0.5)[0]
  print(tokenizer.decode(
      generated_text[len(article):], skip_special_tokens=True))
  print("------------------------------------------------------------ \n")
    

new_article

Suleyman Kerimov <|sep|> The husband of one of the Conservative Party's biggest donors was secretly funded by a Russian oligarch with close ties to President Putin. Leaked files show her husband received $8m (PS6.1m). The money initially came from a politician facing US sanctions due to his closeness to the Kremlin. Her lawyers say the donations are not tainted by Kremlin influence. Advertisement A leak of banks' "suspicious activity reports" - called the FinCEN Files and seen by BBC Panorama - shows Vladimir Chernukhin was sent the money in 2016 from a British Virgin Islands company linked to Suleyman Kerimov. Billionaire Mr Kerimov is a member of the upper chamber of the Russian parliament. He has been under investigation in France over allegations of tax fraud since 2016. Mr Chernukhin, 52, is a former deputy minister of finance under Vladimir Putin, who left Russia for London in 2004 after being sacked by the president. The Chernukhins - one of the UK's most prominent 